In [1]:
import cv2
import numpy as np

video_path = r"C:\Users\basud\Water.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
foreground_output_path = r"C:\Users\basud\foreground_output.mp4"
background_output_path = r"C:\Users\basud\background_output.mp4"

foreground_writer = cv2.VideoWriter(foreground_output_path, fourcc, fps, (frame_width, frame_height))
background_writer = cv2.VideoWriter(background_output_path, fourcc, fps, (frame_width, frame_height))

subtractor = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=400, detectShadows=True)

background_model = None
alpha = 0.01

while True:
    ret, frame = cap.read()
    if not ret:
        break

    fg_mask = subtractor.apply(frame)
    _, fg_mask = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    fg_mask = cv2.erode(fg_mask, kernel, iterations=1)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)
    foreground = cv2.bitwise_and(frame, frame, mask=fg_mask)

    foreground = cv2.convertScaleAbs(foreground, alpha=1.2, beta=20)
    sharpening_kernel = np.array([[0, -1, 0],
                                   [-1, 5, -1],
                                   [0, -1, 0]])
    foreground = cv2.filter2D(foreground, -1, sharpening_kernel)

    if background_model is None:
        background_model = frame.astype(np.float32)
    else:
        cv2.accumulateWeighted(frame, background_model, alpha)

    background = cv2.convertScaleAbs(background_model)
    foreground_writer.write(foreground)
    background_writer.write(background)

    cv2.imshow("Original Frame", frame)
    cv2.imshow("Foreground", foreground)
    cv2.imshow("Background", background)
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
foreground_writer.release()
background_writer.release()
cv2.destroyAllWindows()
print(f"Foreground video saved at: {foreground_output_path}")
print(f"Background video saved at: {background_output_path}")

Foreground video saved at: C:\Users\basud\foreground_output.mp4
Background video saved at: C:\Users\basud\background_output.mp4
